In [ ]:
# Importando bibliotecas necessárias
from sqlalchemy import create_engine, text
import redis

# Configurações do MariaDB (usando o nome do serviço no docker-compose)
mariadb_user = 'epinga_db_user'
mariadb_password = 'epinga_db_password'
mariadb_host = 'mariadb'  # Nome do serviço no docker-compose.yml
mariadb_port = 3306
mariadb_database = 'e_pinga'

# Criando a engine e conectando ao MariaDB dentro do Docker
engine = create_engine(f'mariadb+mariadbconnector://{mariadb_user}:{mariadb_password}@{mariadb_host}:{mariadb_port}/{mariadb_database}')

# Testando a conexão ao MariaDB
try:
    with engine.connect() as connection:
        print(f"Conectado ao banco de dados MariaDB '{mariadb_database}' com sucesso.")
except Exception as e:
    print(f"Erro ao conectar ao MariaDB: {e}")

# Configurações do Redis (usando o nome do serviço no docker-compose)
redis_host = 'redis'
redis_port = 6379
redis_db = 0

# Conectando ao Redis dentro do Docker
r = redis.Redis(host=redis_host, port=redis_port, db=redis_db)

# Testando a conexão ao Redis
try:
    r.ping()
    print("Conexão com o Redis estabelecida.")
except redis.ConnectionError as e:
    print(f"Falha na conexão com o Redis: {e}")


In [ ]:
# Criando as tabelas no MariaDB e as coleções no MongoDB

# Criando tabelas no MariaDB
with engine.connect() as connection:
    connection.execute(text('''
CREATE TABLE IF NOT EXISTS usuario (
    cpf BIGINT PRIMARY KEY,
    nome VARCHAR(200) NOT NULL,
    email VARCHAR(100),
    senhaHash VARCHAR(200) NOT NULL,
    dataNasc DATE,
    cadastro DATE,
    statusFacace CHAR(1)
);

CREATE TABLE IF NOT EXISTS endereco (
    idEndereco INT AUTO_INCREMENT PRIMARY KEY,
    rua VARCHAR(200),
    cidade VARCHAR(100),
    estado VARCHAR(45),
    cep VARCHAR(10),
    pais VARCHAR(45),
    usuario_cpf BIGINT,
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf)
);
    '''))
    print("Tabelas criadas no MariaDB.")

# Criando coleções no MongoDB
collections = ['recomendacoes', 'sessoes', 'produtos_dinamicos']
for collection in collections:
    db.create_collection(collection)
    print(f"Coleção '{collection}' criada no MongoDB.")


In [ ]:
# Inserindo dados nas tabelas e coleções

# Inserindo dados de exemplo no MongoDB
recomendacoes = db['recomendacoes']
recomendacoes.insert_many([
    {
        'recommendation_id': 'rec123',
        'user_id': 'ex188986',
        'items': [
            {
                'item_id': 'item456',
                'nome': 'Cerveja IPA',
                'categoria': 'Cervejas',
                'preco': 15.99
            },
            {
                'item_id': 'item789',
                'nome': 'Vinho Tinto',
                'categoria': 'Vinhos',
                'preco': 45.50
            }
        ],
        'metadata': {
            'timestamp': '2024-04-27T10:00:00Z',
            'interaction_round': 2
        }
    }
])
print("Dados de exemplo inseridos na coleção 'recomendacoes' do MongoDB.")

# Inserindo uma sessão de usuário no Redis
session_key = 'user_session_ex188986'
session_data = {
    'session_id': 'sess123',
    'last_active': '2024-04-27T10:00:00Z'
}
r.set(session_key, json.dumps(session_data))
print(f"Sessão '{session_key}' inserida no Redis.")


### 3.4. Inserção de Dados de Exemplo no MongoDB

In [ ]:
# Inserindo dados de exemplo na coleção 'recomendacoes'
recomendacoes = db['recomendacoes']

recomendacoes.insert_many([
    {
        'recommendation_id': 'rec123',
        'user_id': 'ex188986',
        'items': [
            {
                'item_id': 'item456',
                'nome': 'Cerveja IPA',
                'categoria': 'Cervejas',
                'preco': 15.99
            },
            {
                'item_id': 'item789',
                'nome': 'Vinho Tinto',
                'categoria': 'Vinhos',
                'preco': 45.50
            }
        ],
        'metadata': {
            'timestamp': '2024-04-27T10:00:00Z',
            'interaction_round': 2
        }
    }
])

print("Dados de exemplo inseridos na coleção 'recomendacoes' do MongoDB.")

## 4. Configuração e Inicialização do Redis

### 4.1. Importando Bibliotecas Necessárias

In [ ]:
import redis
import json

### 4.2. Conectando ao Redis

In [ ]:
# Configurações de conexão
redis_host = 'localhost'
redis_port = 6379
redis_db = 0

# Conectando ao Redis
r = redis.Redis(host=redis_host, port=redis_port, db=redis_db)

try:
    r.ping()
    print("Conexão com o Redis estabelecida.")
except redis.ConnectionError:
    print("Falha na conexão com o Redis.")

### 4.3. Inicialização de Dados no Redis

In [ ]:
# Inserindo dados de exemplo no Redis
session_key = 'user_session_ex188986'
session_data = {
    'session_id': 'sess123',
    'last_active': '2024-04-27T10:00:00Z'
}

r.set(session_key, json.dumps(session_data))
print(f"Sessão '{session_key}' inserida no Redis.")

In [ ]:
# Código SQL para criação das tabelas baseadas no diagrama fornecido
create_tables_sql = '''

CREATE TABLE IF NOT EXISTS usuario (
    cpf BIGINT PRIMARY KEY,
    nome VARCHAR(200) NOT NULL,
    email VARCHAR(100),
    senhaHash VARCHAR(200) NOT NULL,
    dataNasc DATE,
    cadastro DATE,
    statusFacace CHAR(1)
);

CREATE TABLE IF NOT EXISTS endereco (
    idEndereco INT AUTO_INCREMENT PRIMARY KEY,
    rua VARCHAR(200),
    cidade VARCHAR(100),
    estado VARCHAR(45),
    cep VARCHAR(10),
    pais VARCHAR(45),
    usuario_cpf BIGINT,
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf)
);

CREATE TABLE IF NOT EXISTS pedido (
    idPedido INT AUTO_INCREMENT PRIMARY KEY,
    usuario_cpf BIGINT,
    carrinho_idCarrinho INT,
    endereco_idEndereco INT,
    dtPedido DATETIME,
    dtPagamento DATETIME,
    statusPedido VARCHAR(45),
    totalPedido FLOAT,
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf),
    FOREIGN KEY (endereco_idEndereco) REFERENCES endereco(idEndereco)
);

CREATE TABLE IF NOT EXISTS itenspedido (
    idItensPedido INT AUTO_INCREMENT PRIMARY KEY,
    pedido_idPedido INT,
    produto_idProduto INT,
    FOREIGN KEY (pedido_idPedido) REFERENCES pedido(idPedido)
);

CREATE TABLE IF NOT EXISTS produto (
    idProduto INT AUTO_INCREMENT PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    logista_cnpj VARCHAR(20),
    categoria_idCategoria INT,
    descricao VARCHAR(500),
    estoque INT,
    preco FLOAT
);

CREATE TABLE IF NOT EXISTS categoriaProduto (
    idCategoria INT AUTO_INCREMENT PRIMARY KEY,
    nome VARCHAR(100),
    descricao VARCHAR(500)
);

CREATE TABLE IF NOT EXISTS logista (
    cnpj VARCHAR(20) PRIMARY KEY,
    razaoSocial VARCHAR(100),
    telefone VARCHAR(45)
);

CREATE TABLE IF NOT EXISTS avaliacao (
    idAvaliacao INT AUTO_INCREMENT PRIMARY KEY,
    usuario_cpf BIGINT,
    produto_idProduto INT,
    estrelas INT,
    comentario VARCHAR(1000),
    dataAvaliacao DATETIME,
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf),
    FOREIGN KEY (produto_idProduto) REFERENCES produto(idProduto)
);

CREATE TABLE IF NOT EXISTS carrinho (
    idCarrinho INT AUTO_INCREMENT PRIMARY KEY,
    usuario_cpf BIGINT,
    dtAbertura DATETIME,
    pagina VARCHAR(45),
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf)
);

CREATE TABLE IF NOT EXISTS itensCarrinho (
    idItensCarrinho INT AUTO_INCREMENT PRIMARY KEY,
    carrinho_idCarrinho INT,
    produto_idProduto INT,
    FOREIGN KEY (carrinho_idCarrinho) REFERENCES carrinho(idCarrinho),
    FOREIGN KEY (produto_idProduto) REFERENCES produto(idProduto)
);

CREATE TABLE IF NOT EXISTS pagamento (
    idPagamento INT AUTO_INCREMENT PRIMARY KEY,
    usuario_cpf BIGINT,
    pedido_idPedido INT,
    dtPagamento DATETIME,
    statusPagamento VARCHAR(45),
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf),
    FOREIGN KEY (pedido_idPedido) REFERENCES pedido(idPedido)
);

CREATE TABLE IF NOT EXISTS logvisitas (
    idLogVis INT AUTO_INCREMENT PRIMARY KEY,
    usuario_cpf BIGINT,
    url VARCHAR(200),
    dtConsulta DATETIME,
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf)
);

CREATE TABLE IF NOT EXISTS logoperacao (
    idLogOperacao INT AUTO_INCREMENT PRIMARY KEY,
    usuario_cpf BIGINT,
    descricao VARCHAR(1000),
    dataOperacao DATETIME,
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf)
);

CREATE TABLE IF NOT EXISTS logcompra (
    idLogCompra INT AUTO_INCREMENT PRIMARY KEY,
    usuario_cpf BIGINT,
    pedido_idPedido INT,
    dtCompra DATETIME,
    total FLOAT,
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf),
    FOREIGN KEY (pedido_idPedido) REFERENCES pedido(idPedido)
);

CREATE TABLE IF NOT EXISTS movimentacaoCarrinho (
    idMovimentacao INT AUTO_INCREMENT PRIMARY KEY,
    usuario_cpf BIGINT,
    carrinho_idCarrinho INT,
    produto_idProduto INT,
    qtProduto INT,
    dataMovimentacao INT,
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf),
    FOREIGN KEY (carrinho_idCarrinho) REFERENCES carrinho(idCarrinho),
    FOREIGN KEY (produto_idProduto) REFERENCES produto(idProduto)
);
'''

# Executando o código para criação das tabelas no MariaDB
with engine.connect() as connection:
    connection.execute(text(create_tables_sql))
    print("Tabelas criadas com sucesso no MariaDB, com base no diagrama fornecido.")

In [ ]:
# Ordem correta: Criar bancos, conectar e, depois, criar tabelas e inserir dados
# Certifique-se de que os serviços MariaDB, MongoDB e Redis estão ativos antes de executar essas operações.

# Conectando ao MariaDB e criando as tabelas
from sqlalchemy import create_engine, text
import pymysql

mariadb_user = 'seu_usuario'
mariadb_password = 'sua_senha'
mariadb_host = 'localhost'
mariadb_port = 3306
mariadb_database = 'e_pinga'

# Criando a engine e conectando
engine = create_engine(f'mysql+pymysql://{mariadb_user}:{mariadb_password}@{mariadb_host}:{mariadb_port}/')
with engine.connect() as connection:
    connection.execute(text(f"CREATE DATABASE IF NOT EXISTS {mariadb_database};"))
    print(f"Banco de dados MariaDB '{mariadb_database}' criado.")

# Selecionando o banco
engine = create_engine(f'mysql+pymysql://{mariadb_user}:{mariadb_password}@{mariadb_host}:{mariadb_port}/{mariadb_database}')
with engine.connect() as connection:
    connection.execute(text('''
CREATE TABLE IF NOT EXISTS usuario (
    cpf BIGINT PRIMARY KEY,
    nome VARCHAR(200) NOT NULL,
    email VARCHAR(100),
    senhaHash VARCHAR(200) NOT NULL,
    dataNasc DATE,
    cadastro DATE,
    statusFacace CHAR(1)
);

CREATE TABLE IF NOT EXISTS endereco (
    idEndereco INT AUTO_INCREMENT PRIMARY KEY,
    rua VARCHAR(200),
    cidade VARCHAR(100),
    estado VARCHAR(45),
    cep VARCHAR(10),
    pais VARCHAR(45),
    usuario_cpf BIGINT,
    FOREIGN KEY (usuario_cpf) REFERENCES usuario(cpf)
);
    '''))
    print("Tabelas criadas no MariaDB.")

# Conectando ao MongoDB
from pymongo import MongoClient
mongodb_host = 'localhost'
mongodb_port = 27017
mongodb_database = 'e_pinga'
client = MongoClient(mongodb_host, mongodb_port)
db = client[mongodb_database]
print(f"Banco de dados MongoDB '{mongodb_database}' conectado.")

# Criando coleções no MongoDB
collections = ['recomendacoes', 'sessoes', 'produtos_dinamicos']
for collection in collections:
    db.create_collection(collection)
    print(f"Coleção '{collection}' criada no MongoDB.")

# Inserindo um exemplo de recomendação no MongoDB
recomendacoes = db['recomendacoes']
recomendacoes.insert_many([
    {
        'recommendation_id': 'rec123',
        'user_id': 'ex188986',
        'items': [
            {
                'item_id': 'item456',
                'nome': 'Cerveja IPA',
                'categoria': 'Cervejas',
                'preco': 15.99
            },
            {
                'item_id': 'item789',
                'nome': 'Vinho Tinto',
                'categoria': 'Vinhos',
                'preco': 45.50
            }
        ],
        'metadata': {
            'timestamp': '2024-04-27T10:00:00Z',
            'interaction_round': 2
        }
    }
])
print("Dados de exemplo inseridos na coleção 'recomendacoes' do MongoDB.")

# Conectando ao Redis
import redis
import json
redis_host = 'localhost'
redis_port = 6379
redis_db = 0
r = redis.Redis(host=redis_host, port=redis_port, db=redis_db)

try:
    r.ping()
    print("Conexão com o Redis estabelecida.")
except redis.ConnectionError:
    print("Falha na conexão com o Redis.")

# Inserindo uma sessão de usuário no Redis
session_key = 'user_session_ex188986'
session_data = {
    'session_id': 'sess123',
    'last_active': '2024-04-27T10:00:00Z'
}
r.set(session_key, json.dumps(session_data))
print(f"Sessão '{session_key}' inserida no Redis.")


## 5. Conclusão

Este notebook demonstrou como configurar e criar os bancos de dados **MariaDB**, **MongoDB** e **Redis** para o sistema e-Pinga. Com essas configurações, o sistema está preparado para armazenar e gerenciar dados estruturados, não estruturados e temporários de forma eficiente e escalável.